In [1]:
import pandas as pd
import pandas_gbq
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr

from time import sleep

import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage

# 경로 변경
os.chdir('/home/owenchoi07/finance_mlops')


# import pydata_google_auth

# pandas_gbq
# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owen-389015'
dataset_id = 'finance_mlops'


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials, 
                         project = credentials.project_id)


# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')

username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')

In [6]:
now = datetime.now()
today_date1 = now.strftime('%Y%m%d')
# start_date = '20180101'
start_date = '20180101'
today_date1 = '20230721'

# 주식 정보

## 티커 리스트

In [3]:
market_list = ['KOSPI', 'KOSDAQ', 'KONEX']

kor_ticker_list_df = pd.DataFrame()
for market_nm in market_list:
    ticker_list = stock.get_market_ticker_list(today_date1, market=market_nm)
    for tickers in ticker_list:
        corp_name = stock.get_market_ticker_name(tickers)
        df = pd.DataFrame({'ticker':tickers,
                           'corp_name':corp_name,
                           'market': market_nm
                          }, index = [0])
        kor_ticker_list_df = pd.concat([kor_ticker_list_df,df])
kor_ticker_list_df = kor_ticker_list_df.reset_index(drop = True)

file_name = 'kor_ticker_list'
# 빅쿼리 데이터 적재
kor_ticker_list_df.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
  project_id=project_id,
  if_exists='replace',
  credentials=credentials)

# Postgresql 적재
kor_ticker_list_df.to_sql(f'{file_name}',if_exists='replace', con=engine,  index=False)
        
kor_ticker_list_df.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')

bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

kor_ticker_list = kor_ticker_list_df['ticker']

100%|██████████| 1/1 [00:00<00:00, 3581.81it/s]


In [7]:

df_raw = stock.get_market_ohlcv(start_date, today_date1, '027410')
df_raw = df_raw.reset_index()
df_raw['ticker'] = '027410'
df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'trading_value', 'price_change_percentage', 'ticker']


In [8]:
df_raw

,date,open,high,low,close,volume,trading_value,price_change_percentage,ticker
0,2018-01-02,15450,15700,15150,15500,763156,11783046100,1.64,027410
1,2018-01-03,15600,15700,15250,15450,647089,9958046500,-0.32,027410
2,2018-01-04,15450,15500,15050,15050,588018,8924957600,-2.59,027410
3,2018-01-05,15000,15200,14500,14600,774720,11418393200,-2.99,027410
4,2018-01-08,14700,15050,14550,14600,504121,7430638300,0.00,027410
...,...,...,...,...,...,...,...,...,...
1365,2023-07-17,3680,3710,3635,3710,37979,139511130,0.68,027410
1366,2023-07-18,3710,3710,3600,3615,111443,405677560,-2.56,027410
1367,2023-07-19,3640,3740,3625,3685,91194,336218850,1.94,027410
1368,2023-07-20,3715,3720,3665,3715,90730,335253255,0.81,027410


## 종목별 주가 정보

In [7]:
for ticker_nm in kor_ticker_list[:2]:
    file_name = 'kor_stock_ohlcv'
    
    try:
        df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
        df_raw = df_raw.reset_index()
        df_raw['ticker'] = ticker_nm
        df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'trading_value', 'price_change_percentage', 'ticker']

        # 빅쿼리 데이터 적재
        df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
          project_id=project_id,
          if_exists='append',
          credentials=credentials)
        
        # Postgresql 적재
        df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print(f'{ticker_nm} success')
    except:
        print(f'{ticker_nm} fail')   


bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)        

100%|██████████| 1/1 [00:00<00:00, 5229.81it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 5461.33it/s]

006840 success


In [33]:
start_date = '20180101'
today_date1 = '20230721'
ticker_nm = '406760'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm
df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'trading_value', 'price_change_percentage', 'ticker']


In [34]:
df_raw

,date,open,high,low,close,volume,trading_value,price_change_percentage,ticker
0,2022-02-25,2080,2100,2080,2090,2983467,6223536400,0.48,406760
1,2022-02-28,2095,2100,2085,2090,246163,515036135,0.00,406760
2,2022-03-02,2090,2105,2085,2095,152902,320541570,0.24,406760
3,2022-03-03,2090,2100,2090,2095,148411,311153110,0.00,406760
4,2022-03-04,2095,2100,2090,2090,58665,122981975,-0.24,406760
...,...,...,...,...,...,...,...,...,...
343,2023-07-17,2215,2220,2195,2210,5202,11473335,-0.23,406760
344,2023-07-18,2210,2230,2205,2225,18665,41498660,0.68,406760
345,2023-07-19,2215,2240,2215,2225,34347,76254150,0.00,406760
346,2023-07-20,2230,2235,2195,2225,12860,28684545,0.00,406760


## 종목별 시가총액

In [8]:
df_raw = stock.get_market_cap('2023-07-11', today_date1, ticker_nm)
df_raw = df_raw.drop(['거래량', '거래대금'], axis = 1)
df_raw.head()

,시가총액,상장주식수
날짜,,
2023-07-11,250246427290,13247561
2023-07-12,264156366340,13247561
2023-07-13,264686268780,13247561
2023-07-14,265613598050,13247561
2023-07-17,272237378550,13247561


In [9]:
for ticker_nm in kor_ticker_list[:2]:
    file_name = 'kor_market_cap'
    
    try:
        df_raw = stock.get_market_cap(start_date, today_date1, ticker_nm)
        df_raw = df_raw.reset_index()
        df_raw['ticker'] = ticker_nm
        df_raw = df_raw.drop(['거래량', '거래대금'], axis = 1)
        df_raw.columns = ['date', 'market_cap', 'outstanding_shares', 'ticker']
        
        # 빅쿼리 데이터 적재
        df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
          project_id=project_id,
          if_exists='append',
          credentials=credentials)
        
        # Postgresql 적재
        df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print(f'{ticker_nm} success')
    except:
        print(f'{ticker_nm} fail')  

bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)        

100%|██████████| 1/1 [00:00<00:00, 3045.97it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 9058.97it/s]

006840 success


## 종목별 DIV/BPS/PER/EPS

In [10]:
for ticker_nm in kor_ticker_list[:2]:
    file_name = 'kor_stock_fundamental'
    
    try:
        df_raw = stock.get_market_fundamental(start_date, today_date1, ticker_nm)
        df_raw = df_raw.reset_index()
        df_raw['ticker'] = ticker_nm
        df_raw.columns = ['date', 'bps', 'per', 'pbr', 'eps', 'div', 'dps', 'ticker']
        
        # 빅쿼리 데이터 적재
        df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
          project_id=project_id,
          if_exists='append',
          credentials=credentials)
        
        # Postgresql 적재
        df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print(f'{ticker_nm} success')
    except:
        print(f'{ticker_nm} fail')   

bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)        

100%|██████████| 1/1 [00:00<00:00, 3901.68it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 3486.54it/s]

006840 success


##  일자별 거래실적 추이 (거래대금)

In [11]:
buy_sell_type_list = ['순매수', '매수', '매도']
for buy_sell_type in buy_sell_type_list:
    for ticker_nm in kor_ticker_list[:1]:
        file_name = 'kor_stock_trading_value_by_investor'
        try:
            df_raw = stock.get_market_trading_value_by_date(start_date, today_date1, 
                                                                             ticker_nm, 
                                                                             detail=True,
                                                                             on = buy_sell_type)
            df_raw = df_raw.reset_index()
            df_raw['ticker'] = ticker_nm
            df_raw['type'] = buy_sell_type
            df_raw.columns = [
                'date', 
                'financial_investment', 'insurance', 'investment', 'private_equity', 'bank','other_finance', 'pension_fund', # 기관합계 
                'other_corporation', # 기타 법인
                'individual',# 개인
                'foreigner', 'other_foreigner', # 외국인 합계
                'total', 
                'ticker', 'type'
            ]

            # 빅쿼리 데이터 적재
            df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
              project_id=project_id,
              if_exists='append',
              credentials=credentials)

            # Postgresql 적재
            df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)            
            
            if not os.path.exists(f'data_crawler/{file_name}.csv'):
                df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
            else:
                df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
            print(f'{ticker_nm} success')
        except:
            print(f'{ticker_nm} fail')    
#     time.sleep(300)


bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

100%|██████████| 1/1 [00:00<00:00, 9039.45it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 2657.99it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 1923.11it/s]

095570 success


## 일자별 거래실적 추이 (거래량)

In [12]:
buy_sell_type_list = ['순매수', '매수', '매도']
for buy_sell_type in buy_sell_type_list:
    for ticker_nm in kor_ticker_list[:1]:
        file_name = 'kor_stock_trading_volume_by_date'
        try:
            df_raw = stock.get_market_trading_volume_by_date(start_date, today_date1, 
                                                                             ticker_nm, 
                                                                             detail=True,
                                                                             on = buy_sell_type)
            df_raw = df_raw.reset_index()
            df_raw['ticker'] = ticker_nm
            df_raw['type'] = buy_sell_type
            df_raw.columns = [
                'date', 
                'financial_investment', 'insurance', 'investment', 'private_equity', 'bank','other_finance', 'pension_fund', # 기관합계 
                'other_corporation', # 기타 법인
                'individual',# 개인
                'foreigner', 'other_foreigner', # 외국인 합계
                'total', 
                'ticker', 'type'
            ]

            # 빅쿼리 데이터 적재
            df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
              project_id=project_id,
              if_exists='append',
              credentials=credentials)

            # Postgresql 적재
            df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)            
            
            if not os.path.exists(f'data_crawler/{file_name}.csv'):
                df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
            else:
                df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
            print(f'{ticker_nm} success')
        except:
            print(f'{ticker_nm} fail')    
#     time.sleep(300)


bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

100%|██████████| 1/1 [00:00<00:00, 8507.72it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 9554.22it/s]


095570 success


100%|██████████| 1/1 [00:00<00:00, 8594.89it/s]

095570 success


In [13]:
df_raw = stock.get_market_trading_volume_by_date(start_date, today_date1, 
                                                                 ticker_nm, 
                                                                 detail=True,
                                                                 on = buy_sell_type)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm
df_raw['type'] = buy_sell_type

# 인덱스 정보

## 인덱스 리스트

In [15]:
kor_index_list_df = pd.DataFrame()
market_list = ['KOSPI', 'KOSDAQ'] 
 
for market_nm in market_list:
    kor_index_list = stock.get_index_ticker_list(market=market_nm)
    for index_codes in kor_index_list:
        index_name = stock.get_index_ticker_name(index_codes)
        df = pd.DataFrame({'index_code':index_codes,
                           'index_code_nm':index_name,
                           'market': market_nm
                          }, index = [0])
        kor_index_list_df = pd.concat([kor_index_list_df,df])
        
kor_index_list_df = kor_index_list_df.reset_index(drop = True)
kor_index_list_df.head()


file_name = 'kor_index_list_df'
# 빅쿼리 데이터 적재
kor_index_list_df.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
  project_id=project_id,
  if_exists='replace',
  credentials=credentials)

# Postgresql 적재
kor_index_list_df.to_sql(f'{file_name}',if_exists='replace', con=engine,  index=False)

kor_index_list_df.head()
kor_index_list_df.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')


bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)


kor_index_code_list  = kor_index_list_df['index_code']

100%|██████████| 1/1 [00:00<00:00, 8594.89it/s]


## 인덱스 OHLCV 조회

In [16]:
for index_code in kor_index_code_list[:2]:
    file_name = 'kor_index_ohlcv'
    
    try:
        df_raw = stock.get_index_ohlcv(start_date, today_date1, index_code)
        df_raw = df_raw.reset_index()
        df_raw['index_code'] = index_code
        df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'trading_value', 'market_cap', 'index_code']

        # 빅쿼리 데이터 적재
        df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
          project_id=project_id,
          if_exists='append',
          credentials=credentials)
        
        # Postgresql 적재
        df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print(f'{index_code} success')
    except:
        print(f'{index_code} fail')   
        
bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)        

100%|██████████| 1/1 [00:00<00:00, 6069.90it/s]


1001 success


100%|██████████| 1/1 [00:00<00:00, 3393.45it/s]

1002 success


## 인덱스 등락률

In [17]:
for index_code in kor_index_code_list[:2]:
    file_name = 'kor_index_code_fundamental'
    
    try:
        df_raw = stock.get_index_fundamental(start_date, today_date1, index_code)
        df_raw = df_raw.reset_index()
        df_raw['index_code'] = index_code
        df_raw.columns = ['date', 'close', 'price_change_percentage', 'per', 'porward_per', 'pbr', 'dividend_yield', 'index_code']

        # 빅쿼리 데이터 적재
        df_raw.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
          project_id=project_id,
          if_exists='append',
          credentials=credentials)
        
        # Postgresql 적재
        df_raw.to_sql(f'{file_name}',if_exists='append', con=engine,  index=False)
        
        if not os.path.exists(f'data_crawler/{file_name}.csv'):
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')
        else:
            df_raw.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='a', header=False)
        
        print(f'{index_code} success')
    except:
        print(f'{index_code} fail')   
        
bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)        

100%|██████████| 1/1 [00:00<00:00, 3228.87it/s]


1001 success


100%|██████████| 1/1 [00:00<00:00, 3751.61it/s]

1002 success


## 인덱스 구성 종목

In [18]:
index_code_info = pd.DataFrame()
for index_code in kor_index_code_list[:2]:
    pdf = stock.get_index_portfolio_deposit_file(str(index_code))
    df = pd.DataFrame({'ticker':pdf,
                       'index_code': str(index_code)})
    index_code_info = pd.concat([index_code_info, df])
index_code_info = index_code_info.reset_index(drop = True)

In [19]:
index_code_info_2  = pd.merge(index_code_info, kor_index_list_df,
        how = 'left',
        on = 'index_code')
        
index_code_info_2.head()

,ticker,index_code,index_code_nm,market
0,005930,1001,코스피,KOSPI
1,373220,1001,코스피,KOSPI
2,000660,1001,코스피,KOSPI
3,207940,1001,코스피,KOSPI
4,051910,1001,코스피,KOSPI


In [20]:
kor_ticker_list_df = pd.read_csv(f'data_crawler/kor_ticker_list.csv')
kor_ticker_list_df.head()

,ticker,corp_name,market
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI


In [21]:
index_code_master  = pd.merge(index_code_info_2, kor_ticker_list_df[['ticker','corp_name']],
        how = 'left',
        on = 'ticker')
        
index_code_master.head()

file_name = 'index_code_master'
# 빅쿼리 데이터 적재
index_code_master.to_gbq(destination_table=f'{project_id}.{dataset_id}.{file_name}',
  project_id=project_id,
  if_exists='replace',
  credentials=credentials)

# Postgresql 적재
index_code_master.to_sql(f'{file_name}',if_exists='replace', con=engine,  index=False)

index_code_master.head()
index_code_master.to_csv(f'data_crawler/{file_name}.csv', index=False, mode='w')


bucket_name = 'finance-mlops'    # 서비스 계정 생성한 bucket 이름 입력
source_file_name = f'data_crawler/{file_name}.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/{file_name}/{file_name}.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

100%|██████████| 1/1 [00:00<00:00, 3666.35it/s]
